In [55]:
import pandas as pd
import numpy as np
import sklearn

import matplotlib.pyplot as plt
import seaborn as sns

# Setup

In [34]:
df = pd.read_csv("data_3_imputed_kalman.csv", parse_dates=['date'])
df = df.drop(df.columns[0], axis=1)

In [39]:
df.columns

Index(['ID', 'date', 'beepo', 'mood', 'arousal', 'valence', 'activity',
       'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication',
       'appCat.entertainment', 'appCat.finance', 'appCat.game',
       'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel',
       'appCat.unknown', 'appCat.utilities', 'appCat.weather', 'mood_cat'],
      dtype='object')

In [44]:
# Define bins and labels
bins = [-float('inf'), 5, 6, float('inf')]
labels = ['-1', '0', '1']

# Categorize the data using cut
df['mood_cat'] = pd.cut(df['mood'], bins=bins, labels=labels, right=True, include_lowest=True)
df['mood_cat']

0       0
1       0
2       1
3       1
4       0
       ..
6071    1
6072    1
6073    1
6074    1
6075    1
Name: mood_cat, Length: 6076, dtype: category
Categories (3, object): ['-1' < '0' < '1']

In [46]:
df['days_since_start'] = (df['date'] - df['date'].min()).dt.days
df['day_of_week'] = df['date'].dt.dayofweek
df_for_cluster = df.drop("date", axis=1)

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaling the variables:

In [47]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_for_cluster)

In [49]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(scaled_features)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


array([0, 2, 2, ..., 1, 1, 1], dtype=int32)

In [50]:
df['Cluster'] = clusters

In [51]:
from sklearn.metrics import silhouette_score

score = silhouette_score(scaled_features, clusters)


In [52]:
score

0.19396817368551897

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# Assuming 'X' is your set of predictors and 'y' is the target variable
forest = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
forest.fit(X, y)

# Selecting features based on importance
importances = forest.feature_importances_
model = SelectFromModel(forest, prefit=True, threshold='median')  # keeping only the top 50% of features
X_reduced = model.transform(X)

# Now X_reduced contains only the most important features